In [1]:
import pandas as pd
order=pd.read_csv("orders.csv")
order.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [2]:
#loading the users data
user=pd.read_json("users.json")
user.head(7)

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
5,6,User_6,Pune,Gold
6,7,User_7,Chennai,Gold


In [3]:
#now checking the sql fie
with open("restaurants.sql","r") as file:
  sqldata=file.read()
sqldata[:500]



"CREATE TABLE restaurants (\nrestaurant_id INT,\nrestaurant_name TEXT,\ncuisine TEXT,\nrating FLOAT);\n\nINSERT INTO restaurants VALUES (1, 'Restaurant_1', 'Chinese', 4.8);\nINSERT INTO restaurants VALUES (2, 'Restaurant_2', 'Indian', 4.1);\nINSERT INTO restaurants VALUES (3, 'Restaurant_3', 'Mexican', 4.3);\nINSERT INTO restaurants VALUES (4, 'Restaurant_4', 'Chinese', 4.1);\nINSERT INTO restaurants VALUES (5, 'Restaurant_5', 'Chinese', 4.8);\nINSERT INTO restaurants VALUES (6, 'Restaurant_6', 'Chinese', 4"

In [4]:
import re
import pandas as pd
rows=re.findall(r'\(([^)]+)\)', sqldata)

data=[]
for r in rows[1:]:
    parts=[x.strip().strip("' ") for x in r.split(",")]
    if len(parts) == 4:
        data.append(parts)

restaurant=pd.DataFrame(
    data,
    columns=["restaurant_id", "restaurant_name", "cuisine", "rating"]
)

restaurant["restaurant_id"] = restaurant["restaurant_id"].astype(int)
restaurant["rating"] = restaurant["rating"].astype(float)

restaurant.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
restaurant["cuisine"].unique()

array(['Chinese', 'Indian', 'Mexican', 'Italian'], dtype=object)

In [6]:


# left join orders and users
orders_users = order.merge(user, on="user_id", how="left")

# left join with restaurants
final_data = orders_users.merge(
    restaurant,
    on="restaurant_id",
    how="left"
)

final_data.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
# saving the final merged dataset
final_data.to_csv("final_food_delivery_dataset.csv", index=False)

In [8]:
#1Q
gold_users= final_data[final_data["membership"] == "Gold"]

gold_city_revenue = gold_users.groupby("city")["total_amount"].sum()

gold_city_revenue

,total_amount
city,
Bangalore,994702.59
Chennai,1080909.79
Hyderabad,896740.19
Pune,1003012.32


In [9]:
#2 Question
avg=final_data.groupby("cuisine")["total_amount"].mean()
avg

,total_amount
cuisine,
Chinese,798.389020
Indian,798.466011
Italian,799.448578
Mexican,808.021344


In [10]:
#3 Question
usertotal=final_data.groupby("user_id")["total_amount"].sum()
highusers=usertotal[usertotal > 1000]
countusers=highusers.count()
countusers

np.int64(2544)

In [11]:
#4 question
final_data["rating_range"] = pd.cut(
    final_data["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"],
    include_lowest=True
)
rating_rev = final_data.groupby("rating_range")["total_amount"].sum()
rating_rev

/tmp/ipython-input-2510556907.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_rev = final_data.groupby("rating_range")["total_amount"].sum()


,total_amount
rating_range,
3.0-3.5,2136772.70
3.6-4.0,1717494.41
4.1-4.5,1960326.26
4.6-5.0,2197030.75


In [12]:
#5
gold_data = final_data[final_data["membership"] == "Gold"]

gold_city_avg = gold_data.groupby("city")["total_amount"].mean()

gold_city_avg

,total_amount
city,
Bangalore,793.223756
Chennai,808.459080
Hyderabad,806.421034
Pune,781.162243


In [13]:
# 6
rest_count = final_data.groupby("cuisine")["restaurant_id"].nunique()
rest_count


,restaurant_id
cuisine,
Chinese,120
Indian,126
Italian,126
Mexican,128


In [14]:
#6
cuisine_rev = final_data.groupby("cuisine")["total_amount"].sum()
cuisine_rev

,total_amount
cuisine,
Chinese,1930504.65
Indian,1971412.58
Italian,2024203.80
Mexican,2085503.09


In [15]:
#7
total_orders = final_data.shape[0]
gold_orders = final_data[final_data["membership"] == "Gold"].shape[0]
gold_percent = round((gold_orders / total_orders) * 100)
gold_percent


50

In [16]:
#8
order_count = final_data.groupby("restaurant_id").size()
few_orders = order_count[order_count < 20]
small_rest_data = final_data[final_data["restaurant_id"].isin(few_orders.index)]
avg_order=small_rest_data.groupby("restaurant_id")["total_amount"].mean()

avg_order.sort_values(ascending=False)

,total_amount
restaurant_id,
294,1040.222308
262,1029.473333
77,1029.180833
193,1026.306667
7,1002.140625
...,...
184,621.828947
498,596.815556
192,589.972857


In [17]:
#9
member_cuisine_revenue = final_data.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False)
member_cuisine_revenue

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [19]:
final_data["order_date"] = pd.to_datetime(final_data["order_date"])
final_data["quarter"] = final_data["order_date"].dt.quarter
q_rev = final_data.groupby("quarter")["total_amount"].sum()
print(q_rev)

quarter
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64


/tmp/ipython-input-2997886880.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_data["order_date"] = pd.to_datetime(final_data["order_date"])


In [20]:
#11
final_data[final_data["membership"] == "Gold"]["order_id"].count()

np.int64(4987)

In [21]:
#12
hyd_rev = final_data[final_data["city"] == "Hyderabad"]["total_amount"].sum()
round(hyd_rev)

1889367

In [22]:
#13
final_data["user_id"].nunique()

2883

In [23]:
#14
gold_orders = final_data[final_data["membership"] == "Gold"]

avg_order_value = gold_orders["total_amount"].mean()
round(avg_order_value, 2)

np.float64(797.15)

In [24]:
#15
high_rating_orders = final_data[final_data["rating"] >= 4.5]
high_rating_orders["order_id"].count()

np.int64(3374)

In [25]:
#16
gold_data = final_data[final_data["membership"] == "Gold"]

top_city = (
    gold_data.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_city

'Chennai'

In [26]:
#16
gold_data[gold_data["city"] == top_city]["order_id"].count()

np.int64(1337)

In [27]:
#19
final_data.shape[0]

10000